In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Simulate 10000 paths for 5 years daily sampled with uniformly distribute vol in [5%,50%]
t = np.linspace(0,5,5*365)
s = np.random.uniform(0.05,0.5,size=10000)
dt = pd.Series(t).diff().values
dz = pd.DataFrame( np.random.normal(size=(len(t), len(s))) * np.sqrt(dt.reshape(-1,1)) * s.reshape(1,-1) , index=t, columns=s).fillna(0.0)
z = dz.cumsum()
z.columns = range(len(z.columns))
z.std().hist(bins=51,density=True)

In [ ]:
# Compute a 10/30 macd normalised by vol (stddiffx) and by range (stdx)
stdx = (z.rolling(10).mean() - z.rolling(30).mean())/z.rolling(365).std()
stdxdiff = (z.rolling(10).mean() - z.rolling(30).mean())/(z.diff().rolling(365).std()/np.sqrt(1./365))
c = z.rolling(365).std().mean().mean()/(z.diff().rolling(365).std()/np.sqrt(1./365)).mean().mean()

In [ ]:
# Marginal and joint distributions of normalisation factor, and of momentum indicator with either normalisation.
# Both are fairly strongly correlated

fig, ax = plt.subplots(2,2, figsize=(15,8), squeeze=False)

zsd = pd.Series(z.rolling(365).std().values.flatten()).dropna()
zsd.hist(bins=51,density=True, alpha=0.4, label="stdx", ax=ax[0,0])
dzsd = pd.Series((dz.rolling(365).std()/np.sqrt(1./365.)).values.flatten()).dropna()
dzsd.hist(bins=51, density=True, alpha=0.4, label="stdxdiff", ax=ax[0,0])
ax[0,0].legend()

_tmp = pd.DataFrame({"stdx" : zsd.values.flatten(), "stdxdiff" : dzsd.values.flatten()}).dropna()
_tmp2 = _tmp.sample(1000).dropna()
ax[1,0].scatter(_tmp2["stdx"], _tmp2["stdxdiff"])

corrsd = _tmp.corr().values[0,1]

_tmp = pd.DataFrame({"stdx" : stdx.values.flatten(), "stdxdiff" : stdxdiff.values.flatten()}).dropna()

corrmom = _tmp.corr().values[0,1]
_tmp2 = _tmp.sample(1000).dropna()
_tmp["stdx"].hist(bins=51, alpha=0.4, label="stdx", density=True, ax=ax[0,1])
(_tmp["stdxdiff"]/c).hist(bins=51, alpha=0.4, label="stdxdiff", density=True, ax=ax[0,1])
ax[1,1].scatter(_tmp2["stdx"], _tmp2["stdxdiff"])

ax[0,0].set_title("Marginal distribution of normalisation factors")
ax[1,0].set_title("Joint distribution of normalisation factors: corr = {}".format(corrsd))
ax[0,1].set_title("Marginal distribution of momentum indicators")
ax[1,1].set_title("Joint distribution of momentum indicators: corr = {}".format(corrmom))

In [ ]:
for i in range(5):
    z_ex = z.sample(1, axis=1).iloc[:,0]
    fig, ax = plt.subplots(1,2, figsize=(18,4))
    z_ex.plot(ax=ax[0], label="x")
    (z_ex.rolling(365).mean() + z_ex.rolling(365).std()).plot(ax=ax[0], color="C1", label="std(x)")
    (z_ex.rolling(365).mean() - z_ex.rolling(365).std()).plot(ax=ax[0], color="C1", label="")
    (z_ex.rolling(365).mean() + z_ex.diff().rolling(365).std()/np.sqrt(1./365)).plot(ax=ax[0], color="C2", label="std(x.diff)")
    (z_ex.rolling(365).mean() - z_ex.diff().rolling(365).std()/np.sqrt(1./365)).plot(ax=ax[0], color="C2", label="")
    ax[0].legend()

    (c*stdx[z_ex.name]).plot(ax=ax[1], color="C1", label="std(x)")
    stdxdiff[z_ex.name].plot(ax=ax[1], color="C2", label="std(x.diff)")
    ax[1].legend()